In [21]:
%load_ext autoreload

%autoreload 2

from __future__ import print_function, division

import cPickle as pickle
from glob import glob
import subprocess
import sys
import string
import os
import random
import numpy as np
from os.path import join

import threebody

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [290]:
dmx_span = tuple(144.3+i*365.25 for i in range(6))
basic_params = dict(files="tuned2_aligned_pulses_1400_nosun2",
                     tzrmjd_middle='auto',
                     parfile="0337_de435.par",
                     fit_pos=True,
                     fit_pm=True,
                     fit_px=True,
                     efac=1,
                     t2_astrometry=True,
                     kopeikin=False,
                     ppn_mode='heavysimple',
                     linear_jumps=True,
                     linear_dm=False,
                     fdn_range=(1,5),
                     priors=('dbeta','dgamma'),
                     toa_mode="pipeline",
                     dmx_span=dmx_span,
                     variable_dm=True,
                     variable_ipm=True)


In [344]:
qsub_script = string.Template("""#!/bin/sh
#PBS -lnodes=1:ppn=1:compute:old
#PBS -N threebody_downhill_$name
#PBS -V
#PBS -o $DIR/pbs.out
#PBS -e $DIR/pbs.err

cd $here
export PATH=/users/aarchiba/.virtualenvs/triplesystem/bin:/users/aarchiba/.local/bin:/usr/local/src/presto/bin:/users/aarchiba/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/bin:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/users/aarchiba/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:.:/users/aarchiba/bin:/opt/local/share/cv/bin:/usr/local/bin:/opt/local/bin:/bin:/usr/bin:/bin:/usr/etc:/usr/sbin:/sbin:/usr/local/sbin:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/bin:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:/usr/lib64/qt-3.3/bin
export PYTHONPATH=

python downhill.py $DIR/fitter_params.pickle $DIR/best_values.pickle \
--method $method --best_parameters $sv > $DIR/stdout.txt
""")
def submit(params, name, method="BOBYQA", start=None):
    # FIXME: handle already-running jobs
    here = os.getcwd()
    DIR = join("downhill-runs",name)
    if not os.path.exists(DIR):
        os.makedirs(DIR)
    with open(join(DIR, "fitter_params.pickle"),"wb") as f:
        pickle.dump(params, f)
    F = threebody.Fitter(**params)
    if start is not None:
        F.best_parameters.update(start)
        sv = join(DIR, "start_values.pickle")
        with open(sv,"wb") as f:
            pickle.dump(F.best_parameters, f)
    else:
        sv = "default"
    for p in F.parameters:
        if p not in F.best_parameters:
            raise ValueError("best parameters don't include %s" % p)
    print("Initial goodness of fit:", F.goodness_of_fit(),
          "/", len(F.mjds))
    if F.goodness_of_fit()>1e6:
        raise ValueError
    s = qsub_script.substitute(**locals())
    P = subprocess.Popen(["qsub","-"],stdin=subprocess.PIPE,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)
    out, err = P.communicate(input=s)
    if P.returncode:
        raise ValueError("qsub returned %s, output:\n%s\nerror:\n%s"
                         % (P.returncode, out, err))
    return out, err

In [410]:
start=None

In [408]:
jobid = "5361"
job_dir = glob(os.path.join("emcee-chains",jobid+".*"))[0]

with open(os.path.join(job_dir,"fitter_params.pickle")) as f:
    fitter_params = pickle.load(f)

F = threebody.Fitter(**fitter_params)
best_so_far = np.load(os.path.join(job_dir,"best_so_far.npy"))
start = F.best_parameters.copy()
for i,p in enumerate(F.parameters):
    start[p] = best_so_far[i]
G = threebody.Fitter(**basic_params)
print("MCMC setup before:", F.goodness_of_fit(), 
      "after:", F.goodness_of_fit(start))
print("basic_params before:", G.goodness_of_fit(),
      "after:", G.goodness_of_fit(start))

MCMC setup before: 36706.9372785 after: 36707.8631494
basic_params before: 36706.9372785 after: 36707.8631494


In [409]:
submit(fitter_params, "MCMC", start=start)

Initial goodness of fit: 36707.8631494 / 27110


('5549.nimrod.cv.nrao.edu\n', '')

In [411]:
p = basic_params.copy()
submit(p,"heavysimple",start=start)
#submit(p,"heavysimple",method="basinhopping",start=start)


Initial goodness of fit: 36706.9372785 / 27110


('5550.nimrod.cv.nrao.edu\n', '')

In [412]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_1400_nosun2"
submit(p,"lband",start=start)


Initial goodness of fit: 36706.9372785 / 27110


('5551.nimrod.cv.nrao.edu\n', '')

In [353]:
p = basic_params.copy()
p["ppn_mode"] = "GR"
submit(p,"GR")
#submit(p,"GR",start=start)
#submit(p,"GR",method="Powell",start=start)
#submit(p,"GR",method="basinhopping",start=start)


Initial goodness of fit: 37361.7615041 / 27138


('4552.nimrod.cv.nrao.edu\n', '')

In [394]:
p = basic_params.copy()
p["ppn_mode"] = "GR"
p["dmx_span"] = 1
p["variable_ipm"] = False
submit(p,"GRdaily")
#submit(p,"GR",start=start)
#submit(p,"GR",method="Powell",start=start)
#submit(p,"GR",method="basinhopping",start=start)


Initial goodness of fit: 34189.8614823 / 27110


('4817.nimrod.cv.nrao.edu\n', '')

In [301]:
p = basic_params.copy()
#p["files"] = "tuned2_aligned_pulses_1400_nosun2"
p["linear_dm"] = True
p["variable_dm"] = False
p["variable_ipm"] = False
submit(p,"dmtesting",start=start)
#submit(p,"heavysimple",method="basinhopping",start=start)
p

Initial goodness of fit: 40509.078393 / 27138


{'dmx_span': (144.3, 509.55, 874.8, 1240.05, 1605.3, 1970.55),
 'efac': 1,
 'fdn_range': (1, 5),
 'files': 'tuned2_aligned_pulses_1400_nosun2',
 'fit_pm': True,
 'fit_pos': True,
 'fit_px': True,
 'kopeikin': False,
 'linear_dm': True,
 'linear_jumps': True,
 'parfile': '0337_de435.par',
 'ppn_mode': 'heavysimple',
 'priors': ('dbeta', 'dgamma'),
 't2_astrometry': True,
 'toa_mode': 'pipeline',
 'tzrmjd_middle': 'auto',
 'variable_dm': False,
 'variable_ipm': False}

## Paper runs

In [385]:
p = basic_params.copy()
submit(p,"primary",start=start)

Initial goodness of fit: 36766.3401074 / 27110


('4809.nimrod.cv.nrao.edu\n', '')

In [413]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_5deg"
submit(p,"lowfreq",start=start)

Initial goodness of fit: 43214.6036375 / 27193


('5552.nimrod.cv.nrao.edu\n', '')

In [387]:
p = basic_params.copy()
p["ppn_mode"] = "newtondelta"
submit(p,"newton",start=start)

Initial goodness of fit: 88926.850906 / 27110


('4811.nimrod.cv.nrao.edu\n', '')

In [414]:
p = basic_params.copy()
p["dmx_span"] = 1
p["variable_ipm"] = False
submit(p,"dailyDM",start=start)

Initial goodness of fit: 34186.4744772 / 27110


('5553.nimrod.cv.nrao.edu\n', '')

In [389]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_5deg_WSRT"
submit(p,"WSRT",start=start)
#submit(p,"WSRT",method="Powell",start=start)
#submit(p,"WSRT",method="basinhopping",start=start)


Initial goodness of fit: 4102.47667649 / 3268


('4813.nimrod.cv.nrao.edu\n', '')

In [390]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_5deg_AO"
submit(p,"AO",start=start)
#submit(p,"AO",method="Powell",start=start)
#submit(p,"AO",method="basinhopping",start=start)


Initial goodness of fit: 9000.36382182 / 5355


('4814.nimrod.cv.nrao.edu\n', '')

In [391]:
p = basic_params.copy()
p["files"] = "tuned2_aligned_pulses_5deg_GBT"
submit(p,"GBT",start=start)
#submit(p,"GBT",method="Powell")
#submit(p,"GBT",method="basinhopping")


Initial goodness of fit: 23396.6605104 / 18487


('4815.nimrod.cv.nrao.edu\n', '')

## Incorporate results into database

In [ ]:
bpd = threebody.load_best_parameter_database()

for DIR in glob(join("downhill-runs","*")):
    name = DIR.split("/")[-1]
    d = pickle.load(open(join(DIR,"fitter_params.pickle")))
    F = threebody.Fitter(**d)
    try:
        p = pickle.load(open(join(DIR,"best_values.pickle")))
    except IOError:
        continue
    newp = F.best_parameters.copy()

    newp.update(p)
    if newp==F.best_parameters:
        print(name, "unchanged")
        continue
    try:
        old = F.goodness_of_fit(F.best_parameters)
    except KeyError:
        old = np.inf
    new = F.goodness_of_fit(newp)
    print(name, "existing best parameters:", old)
    print(name, "best parameters from fitter:", new)

    if new<old:
        bpd = threebody.load_best_parameter_database()
        print(name, "updating")
        bpd[F.bpd_k] = newp
        threebody.save_best_parameter_database(bpd)

WSRT unchanged
AO unchanged
GBT unchanged
heavysimple existing best parameters: 36706.9372785
heavysimple best parameters from fitter: 36706.906731
heavysimple updating
GR unchanged
dmtesting unchanged
MCMC existing best parameters: 36706.906731
MCMC best parameters from fitter: 36707.4220321
lband unchanged
primary existing best parameters: 36706.906731
primary best parameters from fitter: 36763.9494175
lowfreq existing best parameters: 43214.6036375
lowfreq best parameters from fitter: 43214.4184441
lowfreq updating
newton unchanged
